# Random Forest

結合 Bagging &amp; Decision Tree 的特性，做 aggregation of aggregation.

Bagging (Random): reduce variance by voting / average

- for t=1,2,...,T; 利用 bootstrap, 從 D 中取出 N' 筆資料當成 $ \tilde{D_t} $
- 把 $ \tilde{D_t} $ 給 base algorithm: A, 獲得 $ g_t $
- 讓每一個 $ g_t $ 投票獲得 G = $ Uniform( \{ g_t \} ) $

Decition Tree (Forest): large variance, especially if fully-grown

- learn b(x) and split D to $ D_c $ by b(x)
- build $ G_c \leftarrow \ DTree( D_c ) $
- return $ G(x) = \sum_{c=1}^C \big[ b(x) = c \big]_{boolean} G_c(x) $

Random Forest = Bagging + Full-grown C&amp;RT decision tree

- for t=1,2,...,T; 利用 bootstrap, 從 D 中取出 N' 筆資料當成 $ \tilde{D_t} $
- 把 $ \tilde{D_t} $ 給 base algorithm: $ DTree(\tilde{D_t}) $, 獲得 $ g_t $
- 讓每一個 $ g_t $ 投票獲得 G = $ Uniform( \{ g_t \} ) $

Random Forest 的優點:

- highly parallel / efficient to learn
- inherit pros of C&amp;RT 
- eliminate cons of fully-grown tree

### Diversifying by Feature Projection

原來 bagging 的 diversity 來自於 randomly sample N' examples from D

另一個可能性是 : randomly sample d' features from x.

$$
\mathcal{Z} \in \Re^{d'} : \text{a Random Subspace of } \mathcal{X}
$$

re-sample new subspace for each b(x) in C&amp;RT

RF = bagging + random subspace C&amp;RT

### Diversifying by Feature Expansion

上述是將 feature 投影到 natural basis 上 (random subspace)

進一步更可以投影在隨機的 basis 上，就如同將幾個 feature 做了 combination 

RF = bagging + random combination C&amp;RT

### Out-Of-Bag estimation

若資料有 N 筆，做 bootstrapping N' = N, 某一筆資料 $ (x_n,y_n) $ 不在 (out of bag) 用來產生 $ g_t $ 的機率是:

$$
\big( 1 - \frac{1}{N} \big)^N \\
= \frac{1}{ \big( 1 + \frac{1}{N-1} \big)^N } \approx \frac{1}{e}
$$

所以對每一個 $ g_t $ 來說，大概有 0.368 N (1/e) 筆的資料是 OOB, 不在用來產生該 hypothesis 的資料中。

可以使用 OOB 資料來 validate 該 $ g_t $，但很少需要這樣做。

通常會用 OOB 資料來驗證 G, 做法如下:

- 由數個 $ g_i $ 產生 $ G_n^- $, 對這幾個 $ g_i $ 的 OOB 就可以拿來 validate $ G_n^- $
- 然後將 OOB 得到的 Error 做平均求得 $ E_{OOB} $

$$
E_{OOB}(G) = \frac{1}{N} \sum_{n=1}^N err \big( y_n, G_n^-(x_n) \big)
$$

$ E_{OOB} $ : self validation of bagging / RF

也不需要再做 re-training.

## Feature Selection

當有許多的 feature, 其中可能有些是與要做的 prediction 無關的。如何 select 出有用的 feature.

例如 redundant, irrelevant features.

Decision Tree: a rare model with build-in Feature Selection.

### Feature Selection by Importance

importance by linear model

$$
score = w^T x = \sum_{i=1}^d w_i x_i
$$

- intuitive estimate: importance(i) = $ | w_i | $, with some good w
- getting good w: learned from data (linear model)
- non-linear models? often much harder.

### Feature Importance by Permutation Test

如果 feature $ x_i $ 是重要的，放些 random values 到 $ x_i $ 中，會 degrade performance.

用什麼 random values ?

- 如果用某種特定分佈 (Uniform, Gaussian...) 原來的 資料分佈 $ P(x_i) $ 會被影響。
- 用 bootstrap, permutation , 原來的 資料分佈 $ P(x_i) $ 可以大致上保留下來。

### Permutation Test

importance(i) = performance(D) - performance( $ D^p $ )

$ D^p $ 是將第 i 個 feature : $ x_i $, 中的資料順序打亂，其他 feature 資料不變。

### Permutation Test in RF

importance(i) = performance(D) - performance( $ D^p $ )

importance(i) = $ E_{OOB}(G) - E_{OOB}(G^p) $

要獲得 $ G^p $ 比較麻煩，可以照樣用 G，但是用 $ E_{OOB}^p $ (在 OOB 驗證 G 時候，在 feature $ x_i $ 上進行 permutation )

因此變成

importance(i) = $ E_{OOB}(G) - E_{OOB}^p(G) $

實務中，用 RF + permutation + OOB 來做 feature selection 是 efficient + promising.